# <font color= "0C5FAD">Resultado final análisis archivo txt</font>

## Preanálisis

- Se debe añadir al archivo original raw un encabezado para que se tome como referencia del numero de columnas adecuadas en el momento de convertir los datos a un dataFrame de pandas. El formato es el siguiente ```#DD/MM/AAAA, HH:MM a. m. - name surname: ‎archivo.jpg (archivo adjunto) info info info```


## Código para análisis

In [1]:
import pandas as pd
import numpy as np
import re
# Código para crear un dataFrame pandas con el archivo txt raw

df = pd.read_csv("datos_02.txt", sep=r'\s+' ,on_bad_lines='skip')

##################################################################################################################
# Código para limpiar la información de los carácteres "," "$" "#" y espacios en blanco
# Después se identifica el tipo de dato y se hace la conversión dependiendo del tipo correcto int, date, time, str

from datetime import datetime, time, date

def determinar_tipo_y_convertir(elemento):
    #Determina el tipo de dato y convierte el elemento al tipo correcto
    
    if pd.isna(elemento) or str(elemento).strip() == '':
        return "NADA"
    
    # LIMPIAR el elemento - quitar comas y espacios
    elemento_limpio = str(elemento).strip().rstrip(',').replace('$', '').replace('#', '')
    #elemento_limpio = str(elemento).strip().replace('$', '').replace('#', '')
    
    # 1. Verificar si es FECHA (DD/MM/AAAA o D/M/AAAA) - YA LIMPIO
    if re.match(r'^\d{1,2}/\d{1,2}/\d{4}$', elemento_limpio):
        try:
            partes = elemento_limpio.split('/')
            dia, mes, año = int(partes[0]), int(partes[1]), int(partes[2])
            return datetime(año, mes, dia).date()
        except ValueError:
            pass
    # 2. Verificar si es HORA (HH:MMam/pm, HH:MM, etc.)
    # Formato 12h con AM/PM (MODIFICADO)
    patron_12h = re.match(r'^(\d{1,2}):(\d{2})\s*[ \s]*([ap])\.\s*[ \s]*m\.$', elemento_limpio, re.IGNORECASE)
    if patron_12h:
        try:
            hora = int(patron_12h.group(1))
            minuto = int(patron_12h.group(2))
            periodo = patron_12h.group(3).lower()  # 'a' o 'p'
            
            # Convertir de 12h a 24h
            if periodo == 'p' and hora != 12:
                hora += 12
            elif periodo == 'a' and hora == 12:
                hora = 0
            
            if 0 <= hora <= 23 and 0 <= minuto <= 59:
                return time(hora, minuto)
        except ValueError:
            pass
    
    # Formato 24h
    patron_24h = re.match(r'^(\d{1,2}):(\d{2})$', elemento_limpio)
    if patron_24h:
        try:
            hora = int(patron_24h.group(1))
            minuto = int(patron_24h.group(2))
            if 0 <= hora <= 23 and 0 <= minuto <= 59:
                return time(hora, minuto)
        except ValueError:
            pass
    
    # 3. Verificar si es NÚMERO ENTERO (usando elemento_limpio)
    if elemento_limpio.lstrip('-').isdigit() and elemento_limpio not in ['', '-']:
        numero = int(elemento_limpio)
        # Si está entre 30 y 49, convertir a 0
       #if 30 <= numero <= 49:
       #    return 0
      # else:
        return numero
    
    # 4. Si no es ninguno de los anteriores, es TEXTO/NOMBRE (devolver el string original SIN COMA)
    return elemento_limpio

# CÓDIGO PRINCIPAL - UN SOLO BLOQUE
print("=== PROCESANDO DATAFRAME COMPLETO ===")
#######################################################################################################################################
# Crear nuevo DataFrame con los datos convertidos
df_limpio = df.map(determinar_tipo_y_convertir)

##Código para trabajar con el dataFrame limpio. Esto es identificar los datos que corresponden a precios
#y aquellos que corresponden a tallas para separarlos en columnas adicionales.

def extraer_y_separar_enteros(fila, indice_fila, umbral=50):
    enteros_mayores = []
    enteros_menores = []
    columnas_mayores = []
    columnas_menores = []
    
    for columna in df_limpio.columns:
        valor = fila[columna]
        if isinstance(valor, (int, np.integer)) and not isinstance(valor, bool):
            # Clasificar según el umbral
            if valor > umbral:
                enteros_mayores.append(valor)
                columnas_mayores.append(columna)
            else:
                enteros_menores.append(valor)
                columnas_menores.append(columna)
            
            # Reemplazar con cero en el DataFrame original
            df_limpio.at[indice_fila, columna] = 0
    
    return enteros_mayores, enteros_menores, columnas_mayores, columnas_menores

# Aplicar a todas las filas
resultados = []
for i in range(len(df_limpio)):
    fila = df_limpio.iloc[i]
    mayores, menores, cols_mayores, cols_menores = extraer_y_separar_enteros(fila, i)
    resultados.append((mayores, menores, cols_mayores, cols_menores))

# Agregar las nuevas columnas al DataFrame
df_limpio['precios'] = [resultado[0] for resultado in resultados]
df_limpio['tallas'] = [resultado[1] for resultado in resultados]
df_limpio['col_precios'] = [resultado[2] for resultado in resultados]
df_limpio['col_tallas'] = [resultado[3] for resultado in resultados]
#################################################################################################################################
# Línea para exportar el dataFrame a excel
df_limpio.to_excel("df_limpio_02.xlsx", index=False)
#Línea para mostrar el resultado final en consola
df_limpio

=== PROCESANDO DATAFRAME COMPLETO ===


,"DD/MM/AAAA,",HH:MM a. m.,-,name,surname:,‎archivo.jpg,(archivo,adjunto),info,info.1,info.2,precios,tallas,col_precios,col_tallas
0,2024-03-30,09:56:00,-,Lleny,Rodriguez:,<Multimedia,omitido>,NADA,NADA,NADA,NADA,[],[],[],[]
1,,0,NADA,NADA,NADA,NADA,NADA,NADA,NADA,NADA,NADA,[350],[],[HH:MM a. m.],[]
2,2025-03-29,10:33:00,-,Lleny,Rodriguez:,‎IMG-20250329-WA0004.jpg,(archivo,adjunto),NADA,NADA,NADA,[],[],[],[]
3,2025-03-29,10:33:00,-,Lleny,Rodriguez:,0,NADA,NADA,NADA,NADA,NADA,[120],[],[‎archivo.jpg],[]
4,2025-03-29,12:00:00,-,Lleny,Rodriguez:,‎IMG-20250329-WA0022.jpg,(archivo,adjunto),NADA,NADA,NADA,[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,,0,NADA,NADA,NADA,NADA,NADA,NADA,NADA,NADA,[120],[38],"[DD/MM/AAAA,]",[-]
196,2025-04-06,12:10:00,-,Yoli:,‎IMG-20250406-WA0010.jpg,(archivo,adjunto),NADA,NADA,NADA,NADA,[],[],[],[]
197,0,,0,paga,felipe,NADA,NADA,NADA,NADA,NADA,NADA,[110],[36],"[DD/MM/AAAA,]",[-]
198,2025-04-06,12:19:00,-,Lleny,Rodriguez:,‎IMG-20250406-WA0015.jpg,(archivo,adjunto),NADA,NADA,NADA,[],[],[],[]


## Calcular la suma de todos los precios

In [2]:
# Sumar TODOS los enteros de TODAS las listas en la columna
suma_total_mayores = df_limpio['precios'].apply(
    lambda x: sum(x) if x else 0
).sum()

print(f"TOTAL: {suma_total_mayores}")

TOTAL: 21114
